In [4]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle


def get_studies(a_study):
    

    with open(f'{a_study}.pickle', 'rb') as handle:
        studies = pickle.load(handle)

    return studies    
chile = get_studies("studies_chile")
rome  = get_studies("studies_rome")
flor  = get_studies("studies_flor")

In [5]:
plot_all_studies(chile, .8, 0, 'chile')
plot_all_studies(rome, .8, 0, 'rome') 
plot_all_studies(flor, .8, 0, 'flor') 


In [6]:
import warnings
warnings.filterwarnings("ignore")
from utils import *

df_chile = pd.read_parquet("data/df_chile.parquet.gzip")
df_rome  = pd.read_parquet("data/df_rome.parquet.gzip")
df_flor  = pd.read_parquet("data/df_flor.parquet.gzip")

In [12]:
from tqdm import tqdm
from pandas.errors import PerformanceWarning

studies       = flor#rome# chile
df_train      =  df_chile
binary_target = 'Alpha-Fet'
df_test       =  df_flor
Independent_testset = True
filtered_trials = [
                    trial 
                    for study in studies
                    for trial in TwoObjectiveSolutions(study, auc_cutoff=0.8, s_cutoff=0).get_filtered_trials().filtered_trials
                ]

import warnings
from pandas.errors import PerformanceWarning


warnings.filterwarnings("ignore", category=PerformanceWarning)

import copy
AUCs = []
df_ranks_abnormal = pd.DataFrame()
df_ranks_healty = pd.DataFrame()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=PerformanceWarning)
    for i, trial in tqdm(enumerate(copy.deepcopy(filtered_trials)), total=len(filtered_trials)):
        # print(trial.values)

        params_i = trial.params
        seed, kfold_splits, xgb_params = params_i.pop('seed'), params_i.pop('kfold_splits'), params_i

        model_instance = ModelInstance(
            df = df_train, 
            target = binary_target,
            xgb_params = xgb_params,
            kfold_splits = kfold_splits,
            Independent_testset = Independent_testset,
            Independent_testset_df = df_test,
            seed = seed,
            
        )
        AUCs.append(model_instance.get_AUC_on_test_data())
        feature_metrics      = model_instance.get_feature_explanation()
        df_ranks_abnormal[i] = feature_metrics["SHAP_abnormal"]
        df_ranks_healty[i]   = feature_metrics["SHAP_healty"]


100%|██████████| 17/17 [00:08<00:00,  1.97it/s]


In [13]:
import plotly.express as px
import numpy as np

df_ranks_long = df_ranks_abnormal.T
order = df_ranks_long.rank(axis="columns", ascending=False).median(axis="rows").sort_values().index
#df_ranks_long["auc_score"] = AUCs
df = df_ranks_long
import plotly.graph_objects as go

# Calculate the median of each variable and sort the columns in descending order
median_values = df.median().sort_values(ascending=False)
sorted_columns = median_values.index

# Create a list to store the box traces
box_traces = []

# Iterate over each sorted column
for column in sorted_columns:
    # Create a box trace for each variable
    box_trace = go.Box(
        y=df[column],
        name=column,
        marker_color='gray',  # Set the marker color to gray
        showlegend=False  # Hide the legend
    )
    # Add the box trace to the list
    box_traces.append(box_trace)

# Create the layout
layout = go.Layout(
    yaxis=dict(
        title='Importance'
    ),
    xaxis=dict(
        tickangle=45,  # Rotate the x-axis labels by 45 degrees
    ),
    autosize=False,  # Turn off autosizing
    width=800,  # Set the width of the figure
    height=300,  # Set the height of the figure
    margin=dict(
        l=0,  # Set left margin to 0
        r=0,  # Set right margin to 0
        t=0,  # Set top margin to 0
        b=0  # Set bottom margin to 0
    )
)

# Create the figure
fig = go.Figure(data=box_traces, layout=layout)

# Show the figure
fig.show()


In [36]:
import plotly.express as px
import numpy as np

df_ranks_long = df_ranks_abnormal.T
order = df_ranks_long.rank(axis="columns", ascending=False).median(axis="rows").sort_values().index
df_ranks_long["auc_score"] = AUCs

fig_barplot = px.bar(df_ranks_abnormal, barmode="group")

def rank_sort(df_ranks_abnormal: pd.DataFrame) -> pd.Index:
    """This takes the dataframe, ranks its variables, and sorts them by the mean ranking"""
    return df_ranks_abnormal.rank(axis="rows").mean(axis="columns").sort_values(ascending=False).index

order = rank_sort(df_ranks_abnormal) # Converts ranks to an ordered list

def wrapper_pcoorplot(df_ranks_abnormal, dims=order.tolist()):
    """Small wrapper around plotly parallel coordinates plot"""
    fig_ranks = px.parallel_coordinates(
        df_ranks_abnormal.T,
        dimensions=dims,
        color=df_ranks_long["auc_score"],
        # labels=labels_relabel,
        color_continuous_scale='BuGn',
    )

    # Calculate global min and max
    global_min = np.min(df_ranks_abnormal.values)
    global_max = np.max(df_ranks_abnormal.values)

    # Create a new 'range' key for each dimension in the figure
    for dimension in fig_ranks.data[0]['dimensions']:
        dimension['range'] = [global_min, global_max]

    fig_ranks.update_layout(
        yaxis_tickformat=".2f",
        coloraxis_colorbar=dict(title=f"{'AUC'} (test data)"),
    )

    return fig_ranks

# Creates the figure with the first [:15] features according to ranking
fig_abnormal = wrapper_pcoorplot(df_ranks_abnormal, order.tolist()[:10])
fig_abnormal.show()  # Show fig, as this is a notebook


ValueError: Length of values (100) does not match length of index (50)